In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#importing the discharge and microdischarge events for all detectors for run 1 

df = pd.read_excel('magnitude_discharge_bege.xlsx')
df1 = pd.read_excel('magnitude_discharge_coax.xlsx')
df2 = pd.read_excel('magnitude_discharge_icpc.xlsx')
df3 = pd.read_excel('magnitude_discharge_ppc.xlsx')

d = pd.read_excel('magnitude_microdischarge_bege.xlsx')
d1 = pd.read_excel('magnitude_microdischarge_coax.xlsx')
d2 = pd.read_excel('magnitude_microdischarge_icpc.xlsx')
d3 = pd.read_excel('magnitude_microdischarge_ppc.xlsx')

# Set the display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
#combining all events 
df_total = pd.concat([df, df1, df2, df3,d,d1,d2,d3])

In [3]:
#grouping everything by event 
df_total= df_total.groupby('Detector Name')['Event'].unique().to_dict()


In [4]:
import pygama
import pygama.lh5 as lh5
import matplotlib.pyplot as plt
import numpy as np
import h5py
from pygama.dsp.processors import pole_zero
plt.rcParams['figure.figsize'] = [10, 10]
import json
from scipy.stats import zscore
from scipy.optimize import curve_fit

In [5]:
new_raw = f'/unix/legend/wquinn/discharges/raw_files/ '
sto = lh5.Store() # we require this to read 
fw_new1=f'/unix/legend/wquinn/discharges/raw_files/l200-p06-r007-tst-20230726T004658Z_tier_raw.lh5'

In [6]:
f = open('channel_maps.json','r')
cmap = json.load(f)

In [7]:
targets = ["bege","ppc","icpc","coax"]
matching_targets=[]
for i in targets:
    matching_bege = [key for key, value in cmap.items() if "type" in value and value["type"] == i]
    matching_targets.append(matching_bege)

In [8]:
# Character to add
ch = "ch"


all_detectors = [ch + element for sublist in matching_targets for element in sublist]

In [ ]:
tb_data_all = {}  # Dictionary to store tb_data for each detector
wfs_all_all = {}  # Dictionary to store wfs_all for each detector

for j in all_detectors:
    tb_data_all[j] = sto.read_object(f'/{j}/raw', fw_new1)[0]
    wfs_all_all[j] = tb_data_all[j]['waveform']['values'].nda

In [ ]:
#extracting all detectors ie bege ppc icpc and coax 
flat_list = [item for sublist in matching_targets for item in sublist]


In [ ]:
location={}

#extracting the positions of each detector 

for i in flat_list:
    if i in cmap:
        target_value = cmap[i]
        location_info = cmap[i].get('location', {})
        location["ch"+i]=location_info
        
        # Extract 'string' and 'position' from the 'location' dictionary
        string_value = location_info.get('string', None)
        position_value = location_info.get('position', None)

In [ ]:
import pandas as pd


df = pd.DataFrame(columns=['Event', 'Detector name', 'String', 'Position'])

common_values = {}

for key, lst in df_total.items():
    for value in lst:
        if value in common_values:
            common_values[value].append(key)
        else:
            common_values[value] = [key]

for value, keys in common_values.items():
    if len(keys) > 1:
        for j in keys:
            x = j[2:]
            test = cmap[x]['type'] #type of detector
            wfs_all = wfs_all_all[j]
            wf0 = wfs_all[value, :]
            ts = np.arange(0, wf0.shape[0], 1) * 0.8  # micro s

            location_info = location[j]

            string_value = location_info.get('string', None)
            position_value = location_info.get('position', None)

            df = df.append({
                'Event': value,
                'Detector name': j,
                'String': string_value,
                'Position': position_value,
            }, ignore_index=True)

            #plt.plot(ts[:], wf0[:], label=f'{j} which is {test}')  # plot each skipped waveform
            #plt.legend()
        #plt.title(f"The event {value}")
        #plt.show()

#plt.xlabel('Time ($\mu s$)')
#plt.ylabel('Voltage (mV)')


In [22]:
excel_filename = 'all_multi.xlsx'
df.to_excel(excel_filename, index=False)